In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [77]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [78]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

### Read the CSV and Perform Basic Data Cleaning 

In [79]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [80]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

df.tail(5)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
115670,5000.0,13.56%,169.83,MORTGAGE,48000.0,Not Verified,Jan-2019,Current,n,8.28,...,90.5,NaN,0.0,0.0,88613.0,9182.0,0.0,11413.0,N,N
115671,6000.0,6.46%,183.79,MORTGAGE,96000.0,Not Verified,Jan-2019,Current,n,0.31,...,100.0,0.0,1.0,0.0,33500.0,50.0,14900.0,0.0,N,N
115672,16000.0,16.14%,563.62,MORTGAGE,32000.0,Source Verified,Jan-2019,Current,n,20.89,...,93.3,37.5,1.0,0.0,34200.0,17066.0,16400.0,0.0,N,N
115673,16000.0,11.31%,350.36,MORTGAGE,72000.0,Verified,Jan-2019,Current,n,7.02,...,94.3,0.0,1.0,0.0,251486.0,74835.0,23000.0,63090.0,N,N
115674,29250.0,18.94%,757.80,MORTGAGE,65000.0,Verified,Jan-2019,Current,n,29.52,...,100.0,76.9,0.0,0.0,237833.0,58788.0,38800.0,25728.0,N,N


In [81]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

df = df.dropna()

In [82]:
# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
#keeps it as current only and got rid of issued 
df = df.loc[issued_mask]

#convert interest rate to numerical 

df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

In [83]:
#convert the target columns valusto low_risk and high_risk based on their values

x = {'Current': 'low_risk'}
df = df.replace(x)


x = dict.fromkeys(['Late (31-120 Days)', 'Late (16-30 Days)', 'Default', 'In Grace Period'], 'high_risk')
df = df.replace(x)

df.reset_index(inplace=True, drop=True)
df.head(5)

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


## Split the Data into Training and Testing

In [84]:
# Create our features
X = df.drop(columns = 'loan_status')

# Create our target
y = df['loan_status']

In [85]:
#objects are strings 
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 85 columns):
loan_amnt                     68817 non-null float64
int_rate                      68817 non-null float64
installment                   68817 non-null float64
home_ownership                68817 non-null object
annual_inc                    68817 non-null float64
verification_status           68817 non-null object
issue_d                       68817 non-null object
pymnt_plan                    68817 non-null object
dti                           68817 non-null float64
delinq_2yrs                   68817 non-null float64
inq_last_6mths                68817 non-null float64
open_acc                      68817 non-null float64
pub_rec                       68817 non-null float64
revol_bal                     68817 non-null float64
total_acc                     68817 non-null float64
initial_list_status           68817 non-null object
out_prncp                     68817 non-null flo

In [86]:
X = pd.get_dummies(X)

In [87]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 95 columns):
loan_amnt                              68817 non-null float64
int_rate                               68817 non-null float64
installment                            68817 non-null float64
annual_inc                             68817 non-null float64
dti                                    68817 non-null float64
delinq_2yrs                            68817 non-null float64
inq_last_6mths                         68817 non-null float64
open_acc                               68817 non-null float64
pub_rec                                68817 non-null float64
revol_bal                              68817 non-null float64
total_acc                              68817 non-null float64
out_prncp                              68817 non-null float64
out_prncp_inv                          68817 non-null float64
total_pymnt                            68817 non-null float64
total_pymnt_inv        

In [88]:
y = y.apply(lambda x: 0 if x == 'low_risk' else 1)


In [89]:
y.value_counts()

0    68470
1      347
Name: loan_status, dtype: int64

In [90]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1,
                                                       )

X_train.shape

(51612, 95)

In [91]:
# Sclae the data 
from sklearn.preprocessing import StandardScaler
# Create the StandardScaler instance
scaler = StandardScaler()

# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

## Balanced Random Forest Classifier

In [92]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier 
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler

brf_model = BalancedRandomForestClassifier(n_estimators =100, random_state=1)


In [93]:
#fit the model

brf_model = brf_model.fit(X_train_scaled, y_train)

In [94]:
#predictions 

predictions = brf_model.predict(X_test_scaled)

In [108]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)

0.7691539353888617

In [109]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)
# cm_df = pd.DataFrame(
#     cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
# )



array([[14965,  2139],
       [   34,    67]], dtype=int64)

In [110]:
# Print the imbalanced classification report
# Displaying results
# print("Confusion Matrix")
# display(cm_df)
# print(f"Accuracy Score : {bs}")
# print("Classification Report")
print(f"Low risk is scored as 0 & High Risk is assigned value 1")
print(classification_report_imbalanced(y_test, predictions))

Low risk is scored as 0 & High Risk is assigned value 1
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.87      0.66      0.93      0.76      0.59     17104
          1       0.03      0.66      0.87      0.06      0.76      0.57       101

avg / total       0.99      0.87      0.66      0.93      0.76      0.59     17205



In [111]:
# List the features sorted in descending order by feature importance
importances = brf_model.feature_importances_

In [112]:
# List the top 10 most important features
#zip zips up left array and right array X.columns and brf_model feat import 
importances_sorted = sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.08444886105019118, 'total_rec_prncp'),
 (0.06276550792440583, 'total_pymnt_inv'),
 (0.0583991446922582, 'total_rec_int'),
 (0.058074397590724525, 'last_pymnt_amnt'),
 (0.05589516962890926, 'total_pymnt'),
 (0.0345764932940283, 'int_rate'),
 (0.02173197237228757, 'issue_d_Jan-2019'),
 (0.021665114995042582, 'installment'),
 (0.016500213823029363, 'il_util'),
 (0.016317942844767074, 'dti')]

## Easy Ensemble AdaBoost Classifier 

In [103]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

ada_model = EasyEnsembleClassifier(n_estimators =100, random_state=1)


#fit model
ada_model.fit(X_train_scaled, y_train)

#predictions 

predictions_ada = ada_model.predict(X_test_scaled)

In [105]:
# Calculated the balanced accuracy score
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions_ada)

0.931601605553446

In [106]:
# Display the confusion matrix
# Calculated the balanced accuracy score
# bs = balanced_accuracy_score(y_test, predictions_ada)


# Display the confusion matrix
confusion_matrix(y_test, predictions_ada)


array([[16119,   985],
       [    8,    93]], dtype=int64)

In [113]:
# Display the confusion matrix
# print("Confusion Matrix")
# display(cm_df)
# print(f"Accuracy Score : {bs}")
print("Classification Report")
print(classification_report_imbalanced(y_test, predictions_ada))

Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.94      0.92      0.97      0.93      0.87     17104
          1       0.09      0.92      0.94      0.16      0.93      0.87       101

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205



Best balanced Score accuracy: Easy Adaboost is better than RandomForest
    Best Recall Score: Easy Adaboost has higher recall
        Best Geometric Mean Score: Easy Adaboost has higher Geometric mean